# Convert Tracking Results for DanceTrack Metric

## 0. Library

In [23]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from natsort import natsorted
from tqdm.notebook import tqdm

## 1. data load

In [27]:
#予測済みの結果入手
#保存用の実験名
save_path = 'mot20_1113'

pred_txt_dir = '../ultralytics_normal/runs/tracking_results_1115_yolov8_m'
pred_txt_sub = os.listdir(pred_txt_dir)
save_dir = '../ultralytics_normal/Bytetrack_for_dance'
os.makedirs(save_dir,exist_ok = True)

#タスク名
name = 'dance'
pred_txt = natsorted([i for i in pred_txt_sub if name in i])
#部分的にやりたい場合はこっちで調整
pred_txt = pred_txt[19:]
print('exp num = ',len(pred_txt))
print(pred_txt)
#imsize = np.array([1920,1080]

exp num =  6
['tracking_dancetrack0077', 'tracking_dancetrack0079', 'tracking_dancetrack0081', 'tracking_dancetrack0090', 'tracking_dancetrack0094', 'tracking_dancetrack0097']


## 2. write txt fuction

In [28]:
def write_txt(save_path,track_data):
    with open(save_path, 'a') as file:
        file.write(','.join(map(str, track_data)) + '\n')


## 3. exp
- prec txt <0 0.476577 0.643131 0.124049 0.69935 0.879897 1>
- fram center_x center_y width height conf id> 

In [30]:
for exp_name in tqdm(pred_txt):
    #path
    print('---------',exp_name,' convert start-------------')
    exp_dir = os.path.join(pred_txt_dir, exp_name + '/labels')
    image_dir = os.path.join(pred_txt_dir, exp_name + '/00000001.jpg')
    image = cv2.imread(image_dir)
    imsize = image.shape
    print('frame size = ',imsize)
    
    
    
    txt_list = os.listdir(exp_dir)
    
    #txt_list = txt_list[:-1]# リスト内包表記を使用して.ipynbcheckpointを含まないファイルだけを抽出
    txt_list = [file for file in txt_list if not file.endswith('.ipynb_checkpoints')]
    print('frame sum = ',len(txt_list))
    exp_name = exp_name.replace('tracking_','')
    #os.makedirs(os.path.join(save_dir,exp_name),exist_ok = True)
    #print(exp_name)
    #print(len(txt_list))
    
    for num,frame_txt in enumerate(txt_list): 
        #print(frame_txt)
        data = os.path.join(exp_dir,frame_txt)
        df = pd.read_csv(data,sep = ' ',header=None, names=['class','center_x', 'center_y', 'width', 'height','conf','id'])

        for item in zip(df['center_x'], df['center_y'], df['width'], df['height'],df['conf'],df['id']):
            #元の値に戻す
            #print(item)
            center_x = int(item[0]*imsize[1])
            center_y = int(item[1]*imsize[0])
            width = int(item[2]*imsize[1])
            height = int(item[3]*imsize[0])
            
            #なぜかIDがない場合があるので、その場合は補間
            
            id = item[5]
            frame_num = num + 1
            #信頼度を入れる場合はここで指定
            conf = item[4]

            #gtの形に変更
            x1 = center_x - width//2
            y1 = center_y - height//2
            x2 = center_x + width//2
            y2 = center_y + height//2
            #mot
            track_data = [frame_num,id,x1,y1,width,height,conf,-1,-1,-1]

            #save_txt
            save_path = os.path.join(save_dir+'/'+exp_name)
            write_txt(save_path,track_data)

print('finished')
    

  0%|          | 0/6 [00:00<?, ?it/s]

--------- tracking_dancetrack0077  convert start-------------
frame size =  (1080, 1920, 3)
frame sum =  1203


ParserError: Too many columns specified: expected 7 and found 6